<a href="https://colab.research.google.com/github/miki2123j/Amazon-Reviews-Spam-Detection-using-LLM-Model/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning and manupulation

Install any dependencies using pip

In [ ]:
!pip install colorama
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=2b8cdeb825b006fe0254c62c81377a9be501c0e0c9b9bec99319dbccf6c79f38
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Import Packages here

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
import colorama
from colorama import Fore, Style  # makes strings colored
from termcolor import colored
from termcolor import cprint
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_date
from pyspark.sql.functions import regexp_replace
import pyspark.sql.functions as F

In [ ]:
# configuration option to LEGACY to ensure that they are parsed correctly. the show() method on your DataFrame should work.
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")


## Loading in the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("ImportLargeFile").getOrCreate()

# Replace 'your_file_path' with the actual path to your 18GB file
file_path = "/content/drive/MyDrive/archive (3)/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.json"

# Read the file into a DataFrame
df = spark.read.json(file_path)
df_copy = df.select(*df.columns)

# Show the first few rows of the DataFrame
df_copy.show()

+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|{5a1321d5741a2384...|0110400550|Cell_Phones_and_A...|  1.0| [4, 4]|    5.0|Best phone case e...|01 13, 2013| A3HVRXV0LVJN7|        BiancaNicole|               A++++|    1358035200|
|{5a1321d5741a2384...|0110400550|Cell_Phones_and_A...|  0.0| [0, 3]|    1.0|ITEM NOT SENT fro...|01 30, 2013|A1BJGDS0L1IO6I|              cf "t"|     ITEM NOT SENT!!|    1359504000|
|{5a1321d5741a2384...|0110400550|Cell_Phones_and_A...|  1.0| [0, 0]|    5.0|Saw this same 

## Data Preprocessing

In [ ]:
df_copy.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- class: double (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: string (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [ ]:
#checking the unique values of a column

unique_names = df_copy.select("class").distinct()
unique_names.show()

+-----+
|class|
+-----+
|  0.0|
|  1.0|
+-----+



## Check for duplicate values and remove them

In [ ]:
#function that identify duplicate values and remove them from the dataframe
def duplicate_values(df):
    """
    Drops duplicate values from a PySpark dataframe.

    Args:
        df: The PySpark dataframe to check for duplicates.

    Returns:
        None
    """

    print(colored("Duplicate check...", attrs=['bold']), sep='')

    # Check for duplicate values
    duplicate_values = df.drop_duplicates().count()

    # If there are duplicates, drop them
    if duplicate_values > 0:
        df = df.drop_duplicates()  # Remove the 'inplace' keyword argument
        print(duplicate_values, colored(" Duplicates were dropped!"), '\n',
              colored('*'*100, 'red', attrs=['bold']), sep='')
    else:
        print(colored("No duplicates found.", attrs=['bold']), '\n',
              colored('*'*100, 'green', attrs=['bold']), sep='')

In [ ]:
duplicate_values(df_copy)

Duplicate check...
3185845 Duplicates were dropped!
****************************************************************************************************


## Removing special characters

In [ ]:
df_copy = df_copy.withColumn("_id", col("_id").getField("$oid"))
df_copy = df_copy.withColumn("_id", regexp_replace(col("_id"), "[{}]", ""))

In [ ]:
df_copy.show()

+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [4, 4]|    5.0|Best phone case e...|01 13, 2013| A3HVRXV0LVJN7|        BiancaNicole|               A++++|    1358035200|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  0.0| [0, 3]|    1.0|ITEM NOT SENT fro...|01 30, 2013|A1BJGDS0L1IO6I|              cf "t"|     ITEM NOT SENT!!|    1359504000|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [0, 0]|    5.0|Saw this same 

## Identifiying the missing values for each columns

In [ ]:
# Select the columns you want to check for missing values
relevant_columns = [c for c in df_copy.columns]

# Count the missing values in each column and display the result
df_copy.select(*[sum(col(c).isNull().cast("int")).alias(c) for c in relevant_columns]).show()


+---+----+--------+-----+-------+-------+----------+----------+----------+------------+-------+--------------+
|_id|asin|category|class|helpful|overall|reviewText|reviewTime|reviewerID|reviewerName|summary|unixReviewTime|
+---+----+--------+-----+-------+-------+----------+----------+----------+------------+-------+--------------+
|  0|   0|       0|    0|      0|      0|         0|         0|         0|       35568|      0|             0|
+---+----+--------+-----+-------+-------+----------+----------+----------+------------+-------+--------------+



## Cleaning missing values

In [ ]:
3# A function that fills null values of a column with given value
def fill_null_values(data, new_value, column):
  data = data.na.fill(new_value,[column])
  return data


In [ ]:
#filling null value of 'label' column with a constant 0
#df_copy = fill_null_values(df_copy, 0, 'label')

In [ ]:
#filling null value of 'reviewername' column with a constant "Unknown"
df_copy = fill_null_values(df_copy, 'Unknown', 'reviewerName')

In [ ]:
df_copy.show()

+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [4, 4]|    5.0|Best phone case e...|01 13, 2013| A3HVRXV0LVJN7|        BiancaNicole|               A++++|    1358035200|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  0.0| [0, 3]|    1.0|ITEM NOT SENT fro...|01 30, 2013|A1BJGDS0L1IO6I|              cf "t"|     ITEM NOT SENT!!|    1359504000|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [0, 0]|    5.0|Saw this same 

## Converting dtypes

In [ ]:
# A function that converts a given string dframe column to date dframe column
def convert_to_date(data, column_name):
  data = data.withColumn(column_name, to_date(column_name, "MM dd, yyyy"))
  return data

In [ ]:
df_copy = convert_to_date(df_copy, 'reviewTime')

In [ ]:
df_copy.printSchema()

root
 |-- _id: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- category: string (nullable = true)
 |-- class: double (nullable = true)
 |-- helpful: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- reviewTime: date (nullable = true)
 |-- reviewerID: string (nullable = true)
 |-- reviewerName: string (nullable = false)
 |-- summary: string (nullable = true)
 |-- unixReviewTime: long (nullable = true)



In [ ]:
df_copy.show(5)

+--------------------+----------+--------------------+-----+-------+-------+--------------------+----------+--------------+----------------+---------------+--------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText|reviewTime|    reviewerID|    reviewerName|        summary|unixReviewTime|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+----------+--------------+----------------+---------------+--------------+
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [4, 4]|    5.0|Best phone case e...|2013-01-13| A3HVRXV0LVJN7|    BiancaNicole|          A++++|    1358035200|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  0.0| [0, 3]|    1.0|ITEM NOT SENT fro...|2013-01-30|A1BJGDS0L1IO6I|          cf "t"|ITEM NOT SENT!!|    1359504000|
|5a1321d5741a2384e...|0110400550|Cell_Phones_and_A...|  1.0| [0, 0]|    5.0|Saw this same cas...|2012-11-22| A1YX2RBMS1L9L|    Andrea Busch|

## Dropping irrelvant columns

In [ ]:
df_copy = df_copy.drop('asin', 'helpful', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime')


In [ ]:
df_copy.show()

+--------------------+--------------------+-----+-------+--------------------+----------+
|                 _id|            category|class|overall|          reviewText|reviewTime|
+--------------------+--------------------+-----+-------+--------------------+----------+
|5a1321d5741a2384e...|Cell_Phones_and_A...|  1.0|    5.0|Best phone case e...|2013-01-13|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  0.0|    1.0|ITEM NOT SENT fro...|2013-01-30|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  1.0|    5.0|Saw this same cas...|2012-11-22|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  1.0|    5.0|case fits perfect...|2013-07-18|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  1.0|    4.0|I got this for my...|2013-03-20|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  1.0|    5.0|This case is extr...|2012-07-14|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  0.0|    3.0|The case is good,...|2012-12-20|
|5a1321d5741a2384e...|Cell_Phones_and_A...|  0.0|    1.0|The case pictured...|2012-08-14|
|5a1321d57

In [ ]:
out_put_path = "/content/drive/MyDrive/Colab Notebooks/cleaned data"
df_copy.coalesce(1).write.json(out_put_path, mode= "overwrite")
